In [8]:
torch.Tensor()

tensor([])

In [9]:
torch.Tensor([1,2,3])

tensor([1., 2., 3.])

In [10]:
torch.Tensor([1,2,3]).bool

<function Tensor.bool>

In [11]:
torch.Tensor([1,2,3]).bool()

tensor([True, True, True])

In [12]:
def getsample(n=50):
  labels = np.random.randint( 2, size=(n,1,1))
  idxs = np.random.randint(50, size=n)
  xn = np.random.randint(24, size=n)
  x = train_org[idxs, xn]
  yn = np.random.randint(24, size=n)
  # y = np.where(labels, train_org[idxs, yn], train_forg[idxs, yn])

  y = torch.where(torch.Tensor(labels).bool(), train_org[idxs, yn], train_forg[idxs, yn])
  return torch.Tensor(x), torch.Tensor(y), torch.Tensor(labels.flatten(), device='cuda')

x,y,labels = getsample(10)


x.shape # torch.Size([10, 200, 300])

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [13]:
torch.Tensor.to('cuda')

TypeError: descriptor 'to' for 'torch._C.TensorBase' objects doesn't apply to a 'str' object

In [14]:
torch.Tensor([1,2,3]).to('cuda')

tensor([1., 2., 3.], device='cuda:0')

In [15]:
torch.Tensor([1,2,3]).to(torch.bool, 'cuda')

TypeError: to() received an invalid combination of arguments - got (torch.dtype, str), but expected one of:
 * (torch.device device, torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (Tensor tensor, bool non_blocking, bool copy, *, torch.memory_format memory_format)


In [16]:
torch.Tensor([1,2,3]).to(torch.bool, dtype='cuda')

TypeError: to() received an invalid combination of arguments - got (torch.dtype, dtype=str), but expected one of:
 * (torch.device device, torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (torch.dtype dtype, bool non_blocking, bool copy, *, torch.memory_format memory_format)
 * (Tensor tensor, bool non_blocking, bool copy, *, torch.memory_format memory_format)


In [17]:
torch.Tensor([1,2,3]).to('cuda', torch.bool)

tensor([True, True, True], device='cuda:0')

In [18]:
def getsample(n=50):
  labels = np.random.randint( 2, size=(n,1,1))
  idxs = np.random.randint(50, size=n)
  xn = np.random.randint(24, size=n)
  x = train_org[idxs, xn]
  yn = np.random.randint(24, size=n)
  # y = np.where(labels, train_org[idxs, yn], train_forg[idxs, yn])

  y = torch.where(torch.Tensor(labels).to('cuda', torch.bool), train_org[idxs, yn], train_forg[idxs, yn])
  return torch.Tensor(x), torch.Tensor(y), torch.Tensor(labels.flatten(), device='cuda')

x,y,labels = getsample(10)


x.shape # torch.Size([10, 200, 300])

RuntimeError: legacy constructor expects device type: cpu but device type: cuda was passed

In [19]:
def getsample(n=50):
  labels = np.random.randint( 2, size=(n,1,1))
  idxs = np.random.randint(50, size=n)
  xn = np.random.randint(24, size=n)
  x = train_org[idxs, xn]
  yn = np.random.randint(24, size=n)
  y = np.where(labels, train_org[idxs, yn], train_forg[idxs, yn])

  # y = torch.where(torch.Tensor(labels).to('cuda', torch.bool), train_org[idxs, yn], train_forg[idxs, yn])
  return torch.Tensor(x), torch.Tensor(y), torch.Tensor(labels.flatten(), device='cuda')

x,y,labels = getsample(10)


x.shape # torch.Size([10, 200, 300])

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

Connected to Python 3.10.12

In [1]:
def getsample(n=50):
  labels = np.random.randint( 2, size=(n,1,1))
  idxs = np.random.randint(50, size=n)
  xn = np.random.randint(24, size=n)
  x = train_org[idxs, xn]
  yn = np.random.randint(24, size=n)
  y = np.where(labels, train_org[idxs, yn], train_forg[idxs, yn])

  # y = torch.where(torch.Tensor(labels).to('cuda', torch.bool), train_org[idxs, yn], train_forg[idxs, yn])

  return torch.Tensor(x), torch.Tensor(y), torch.Tensor(labels.flatten(), device='cuda')

x,y,labels = getsample(10)


x.shape # torch.Size([10, 200, 300])

NameError: name 'np' is not defined

In [2]:
import os
import torch
from torch import nn
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import tqdm

In [3]:
maxdims = [576, 672]
datapath = '/shared/datasets/signatures/signatures'

def loadimage(path):
  img = Image.open(path).convert('L')
  img = np.array(img)
  img = img[:maxdims[0], :maxdims[1]]
  img = np.pad(img, ((0, maxdims[0] - img.shape[0]), (0, maxdims[1] - img.shape[1])))
  return img

def readpath(path):
  collection = {}
  t = tqdm.tqdm(os.listdir(path), desc=f'loading {path}')
  for img_path in t:
    if not img_path.endswith('.png'): continue
    id = img_path.split('_')[1]
    collection.setdefault(id, []).append(loadimage(path+'/'+img_path))
  res = np.array([np.array(collection[key]) for key in sorted(collection.keys())], dtype=np.float32) / 255.0
  return torch.nn.functional.adaptive_avg_pool2d(torch.Tensor(res), (200, 300),).to('cuda')

org = readpath(datapath+'/full_org')
forg = readpath(datapath+'/full_forg')

train_org = org[:50]
train_forg = forg[:50]
test_org = org[50:]
test_forg = forg[50:]

loading /shared/datasets/signatures/signatures/full_org: 100%|██████████| 1321/1321 [00:01<00:00, 743.65it/s]
loading /shared/datasets/signatures/signatures/full_forg: 100%|██████████| 1321/1321 [00:01<00:00, 717.92it/s]


In [4]:
def getsample(n=50):
  labels = np.random.randint( 2, size=(n,1,1))
  idxs = np.random.randint(50, size=n)
  xn = np.random.randint(24, size=n)
  x = train_org[idxs, xn]
  yn = np.random.randint(24, size=n)
  y = np.where(labels, train_org[idxs, yn], train_forg[idxs, yn])

  # y = torch.where(torch.Tensor(labels).to('cuda', torch.bool), train_org[idxs, yn], train_forg[idxs, yn])

  return torch.Tensor(x), torch.Tensor(y), torch.Tensor(labels.flatten(), device='cuda')

x,y,labels = getsample(10)


x.shape # torch.Size([10, 200, 300])

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [5]:
def getsample(n=50):
  labels = np.random.randint( 2, size=(n,1,1))
  idxs = np.random.randint(50, size=n)
  xn = np.random.randint(24, size=n)
  x = train_org[idxs, xn]
  yn = np.random.randint(24, size=n)
  # y = np.where(labels, train_org[idxs, yn], train_forg[idxs, yn])

  y = torch.where(torch.Tensor(labels).to('cuda', torch.bool), train_org[idxs, yn], train_forg[idxs, yn])

  return torch.Tensor(x), torch.Tensor(y), torch.Tensor(labels.flatten(), device='cuda')

x,y,labels = getsample(10)


x.shape # torch.Size([10, 200, 300])

RuntimeError: legacy constructor expects device type: cpu but device type: cuda was passed

In [6]:
def getsample(n=50):
  labels = np.random.randint( 2, size=(n,1,1))
  idxs = np.random.randint(50, size=n)
  xn = np.random.randint(24, size=n)
  x = train_org[idxs, xn]
  yn = np.random.randint(24, size=n)
  # y = np.where(labels, train_org[idxs, yn], train_forg[idxs, yn])

  labels = torch.tensor(labels, device='cuda')
  print(labels)
  y = torch.where(torch.Tensor(labels).to('cuda', torch.bool), train_org[idxs, yn], train_forg[idxs, yn])

  return torch.Tensor(x), torch.Tensor(y), torch.Tensor(labels.flatten(), device='cuda')

x,y,labels = getsample(10)


x.shape # torch.Size([10, 200, 300])

tensor([[[0]],

        [[0]],

        [[0]],

        [[0]],

        [[0]],

        [[0]],

        [[1]],

        [[0]],

        [[1]],

        [[1]]], device='cuda:0')


RuntimeError: Legacy tensor constructor of the form torch.Tensor(tensor, device=device) is not supported.  Use torch.tensor(...) or torch.as_tensor(...) instead.

In [7]:
def getsample(n=50):
  labels = np.random.randint( 2, size=(n,1,1))
  idxs = np.random.randint(50, size=n)
  xn = np.random.randint(24, size=n)
  x = train_org[idxs, xn]
  yn = np.random.randint(24, size=n)
  # y = np.where(labels, train_org[idxs, yn], train_forg[idxs, yn])

  labels = torch.tensor(labels, device='cuda')
  print(labels)
  y = torch.where(labels, train_org[idxs, yn], train_forg[idxs, yn])

  return torch.Tensor(x), torch.Tensor(y), torch.Tensor(labels.flatten(), device='cuda')

x,y,labels = getsample(10)


x.shape # torch.Size([10, 200, 300])

tensor([[[1]],

        [[0]],

        [[0]],

        [[0]],

        [[0]],

        [[0]],

        [[0]],

        [[1]],

        [[1]],

        [[0]]], device='cuda:0')


RuntimeError: where expected condition to be a boolean tensor, but got a tensor with dtype Long

In [8]:
def getsample(n=50):
  labels = np.random.randint( 2, size=(n,1,1))
  idxs = np.random.randint(50, size=n)
  xn = np.random.randint(24, size=n)
  x = train_org[idxs, xn]
  yn = np.random.randint(24, size=n)
  # y = np.where(labels, train_org[idxs, yn], train_forg[idxs, yn])

  labels = torch.tensor(labels, dtype=bool, device='cuda')
  print(labels)
  y = torch.where(labels, train_org[idxs, yn], train_forg[idxs, yn])

  return torch.Tensor(x), torch.Tensor(y), torch.Tensor(labels.flatten(), device='cuda')

x,y,labels = getsample(10)


x.shape # torch.Size([10, 200, 300])

tensor([[[ True]],

        [[ True]],

        [[False]],

        [[ True]],

        [[ True]],

        [[ True]],

        [[False]],

        [[False]],

        [[ True]],

        [[ True]]], device='cuda:0')


RuntimeError: Legacy tensor constructor of the form torch.Tensor(tensor, device=device) is not supported.  Use torch.tensor(...) or torch.as_tensor(...) instead.

In [9]:
def getsample(n=50):
  labels = np.random.randint( 2, size=(n,1,1))
  idxs = np.random.randint(50, size=n)
  xn = np.random.randint(24, size=n)
  x = train_org[idxs, xn]
  yn = np.random.randint(24, size=n)
  # y = np.where(labels, train_org[idxs, yn], train_forg[idxs, yn])

  labels = torch.tensor(labels, dtype=bool, device='cuda')
  print(labels)
  y = torch.where(labels, train_org[idxs, yn], train_forg[idxs, yn])

  return torch.Tensor(x), torch.Tensor(y), labels

x,y,labels = getsample(10)


x.shape # torch.Size([10, 200, 300])

tensor([[[ True]],

        [[ True]],

        [[ True]],

        [[ True]],

        [[False]],

        [[False]],

        [[ True]],

        [[False]],

        [[ True]],

        [[ True]]], device='cuda:0')


torch.Size([10, 200, 300])

In [10]:
def getsample(n=50):
  labels = np.random.randint( 2, size=(n,1,1))
  idxs = np.random.randint(50, size=n)
  xn = np.random.randint(24, size=n)
  x = train_org[idxs, xn]
  yn = np.random.randint(24, size=n)
  # y = np.where(labels, train_org[idxs, yn], train_forg[idxs, yn])

  labels = torch.tensor(labels, dtype=bool, device='cuda')

  y = torch.where(labels, train_org[idxs, yn], train_forg[idxs, yn])
  print(type(y))


  return torch.Tensor(x), torch.Tensor(y), labels

x,y,labels = getsample(10)


x.shape # torch.Size([10, 200, 300])

<class 'torch.Tensor'>


torch.Size([10, 200, 300])

In [11]:
def getsample(n=50):
  labels = np.random.randint( 2, size=(n,1,1))
  idxs = np.random.randint(50, size=n)
  xn = np.random.randint(24, size=n)
  x = train_org[idxs, xn]
  yn = np.random.randint(24, size=n)
  # y = np.where(labels, train_org[idxs, yn], train_forg[idxs, yn])

  labels = torch.tensor(labels, dtype=bool, device='cuda')

  y = torch.where(labels, train_org[idxs, yn], train_forg[idxs, yn])

  return x, y, labels

x,y,labels = getsample(10)


x.shape # torch.Size([10, 200, 300])

torch.Size([10, 200, 300])

In [12]:
x.dtype

torch.float32

In [13]:
x.device

device(type='cuda', index=0)